## 1. Cargamos los datos a Preprocesar y limpiar (desde el BlobStorage)

In [0]:
#Librerias que se usaran
import pandas as pd  # Libreria para administrar tablas, y realizar trabajos con distintas formas de tablas o dataframes
import numpy as np   # Libreria para poder hacer operaciones matematicas y matriciales
import matplotlib.pyplot as plt # Libreria para realizar graficos 

#Configuramos pandas para que podamos vizualizar todas las columnas y filas la estadistica descriptiva de todas las variables
pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', None)
#Configuramos pandas para que lanze valores con una precision de hasta 6 decimales
pd.set_option('display.float_format', '{:.6f}'.format)

In [0]:
#Librerias necesarias para usar el Azure DataLake y DataBrinks
import pandas as pd
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient
import io

# 2. Obtener conection Azure DataLake,interfaz de Azure(Claves de acceso: Key1) (Debes comentar esta variable Si NO deja hacer COMMIT DEL 
# CODIGO EN GIT,GITHUB)
# connection_string = 'DefaultEndpointsProtocol=https;AccountName=datalakemlopsd4m;AccountKey=iWT8t74/#XlqcqoR03keDVtFZPzr0PB9zDffMPaLWMUBIAjUww8uYAVkc9xRkcBtvTmUHKBvd1sB3+ASt6mGgcQ==;EndpointSuffix=core.windows.net'

# 3. Conectar al Blob Storage de Azure
blob_service_client = BlobServiceClient.from_connection_string(connection_string)

# 4. Identificamos el nombre del contenedor(container_name) y nombre del archivo(definir blob_name) en el Blob Storage
container_name = "raw/proyectocongestion_raw/fuentedatos_consolidado/"
blob_name = "datos_raw_shougang_c4m_h4m.csv"

# 5. Obtener el blob_client
blob_client = blob_service_client.get_blob_client(container=container_name, blob=blob_name)

# 6. Leer el contenido del blob como texto
blob_data = blob_client.download_blob().content_as_text()

# Especificar tipos de datos al leer el archivo CSV
column_types = {
    'instant_date_t': 'datetime64[ns]',
    'Event Date' : 'datetime64[ns]'
}

# 7. Leer el archivo CSV en un DataFrame de Pandas desde el texto
datos = pd.read_csv(io.StringIO(blob_data), parse_dates=['instant_date_t', 'Event Date'])

In [0]:
datos.dtypes

In [0]:
datos['isunderground'].value_counts()

In [0]:
datos[["instant_date_t", "Event Date"]].head(5)

In [0]:
datos["eq_id"].unique()

### *1.2 Obtenemos la dimension de los datos*
- Identificaremos la cantidad de filas y columnas que tienen nuestros datos

In [0]:
import plotly.express as px

# Obtiene las dimensiones de tu conjunto de datos
num_filas, num_columnas = datos.shape

# Crea un DataFrame con los datos de las dimensiones
df_dimension = pd.DataFrame({'Dimensión': ['Filas', 'Columnas'], 'Valor': [num_filas, num_columnas]})

# Crea un gráfico de pastel interactivo con etiquetas
fig = px.pie(df_dimension, names='Dimensión', values='Valor', title='Dimensiones del Conjunto de Datos')

# Agrega etiquetas y porcentajes
fig.update_traces(textinfo='label+percent')

# Muestra el gráfico interactivo
fig.show()

### *1.3 Identificacion de Variables Categoricas y Numericas*
- Pasaremos a identificar cuantas variables de nuestros datos son numericas y cuantas categoricas

In [0]:
import matplotlib.pyplot as plt

# Verificamos que tipos de datos estaremos trabajando para el análisis
tipos_de_datos = datos.dtypes

variables_numericas = tipos_de_datos[tipos_de_datos != 'object']
variables_categoricas = tipos_de_datos[tipos_de_datos == 'object']

# Obtiene la cantidad de variables numéricas y categóricas
num_variables_numericas = len(variables_numericas)
num_variables_categoricas = len(variables_categoricas)

# Etiquetas para las categorías
categorias = ['Variables Numéricas', 'Variables Categóricas']

# Valores para las categorías
valores = [num_variables_numericas, num_variables_categoricas]

# Crea un gráfico de barras
plt.figure(figsize=(10, 6))
bars = plt.bar(categorias, valores, color=['blue', 'green'])

# Añade etiquetas en las barras
for bar, valor in zip(bars, valores):
    plt.text(bar.get_x() + bar.get_width() / 2, bar.get_height() + 1, str(valor), ha='center', va='bottom')

# Añade etiquetas y título
plt.ylabel('Número de Variables')
plt.title('Tipos de Datos en el Conjunto de Datos')

# Muestra el gráfico
plt.show()

Tambien identificamos cuales serian esas Variables Categoricas y numericas

In [0]:
import pandas as pd
import plotly.express as px

# Supongamos que tienes un DataFrame llamado 'datos'
tipos_de_datos = datos.dtypes

variables_numericas = tipos_de_datos[tipos_de_datos != 'object']
variables_categoricas = tipos_de_datos[tipos_de_datos == 'object']

# Crea un DataFrame con nombres de variables y tipos de datos
df = pd.DataFrame({'Variable': datos.columns, 'Tipo de Dato': ['Numérica' if col in variables_numericas else 'Categórica' for col in datos.columns]})

# Crea un gráfico interactivo
fig = px.bar(df, x='Variable', color='Tipo de Dato', title='Tipos de Variables')
fig.show()

#### Obtener equipos de camión únicos y sus nombres único

In [0]:
# Obtener equipos de camión únicos y sus nombres únicos
equipos_camiones = datos.loc[datos["eq_id"].notnull(), ["eq_id", "nombre"]] \
    .drop_duplicates(subset="eq_id")
print(equipos_camiones)

#### Obtener por equipos, la velocidad en promedio

In [0]:
datos.groupby('eq_id')['speed_t'].mean()

#### Extraer year, mes,dia,Hora, minuto

In [0]:
# Extraer componentes de fecha y hora
datos['year'] = datos['instant_date_t'].dt.year
datos['mes'] = datos['instant_date_t'].dt.month
datos['dia'] = datos['instant_date_t'].dt.day
datos['hora'] = datos['instant_date_t'].dt.hour
datos['minuto'] = datos['instant_date_t'].dt.minute

In [0]:
datos.head()

#### Obtener por equipos, y por horas, el promedio de la velocidad

In [0]:
import matplotlib.pyplot as plt

# Agrupar por id_equipo y hora, calcular el promedio de velocidad
promedios_velocidad = datos.groupby(['eq_id', 'hora'])['speed_t'].mean().reset_index()

# Crear el gráfico de líneas
plt.figure(figsize=(10, 6))
for id_equipo, group in promedios_velocidad.groupby('eq_id'):
    plt.plot(group['hora'], group['speed_t'], label=f'Equipo {id_equipo}')

# Añadir etiquetas y título
plt.xlabel('Hora')
plt.ylabel('Promedio de Velocidad')
plt.title('Promedio de Velocidad por Hora y Equipo')
plt.legend()
plt.grid(True)

# Mostrar el gráfico
plt.xticks(rotation=45)  # Rotar las etiquetas del eje x para una mejor legibilidad
plt.gca().xaxis.set_major_locator(plt.MultipleLocator(3))  # Mostrar cada 3 horas en el eje x
plt.show()

In [0]:
# Extraer la hora y crear una nueva columna para almacenarla
datos['hora'] = datos['instant_date_t'].dt.hour

# Agrupar por id_equipo y hora, calcular el promedio de velocidad
datos.groupby(['eq_id', 'hora'])['speed_t'].mean()

### *1.4 Identificacion de Valores Nulos*
- Pasaremos a identificar si en nuestros datos tenemos valores Nulos, y si fuese el caso, cuanto porcentaje de valores Nulos tendrian cada variable del Dataset

In [0]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Supongamos que tienes un DataFrame llamado datos
valores_nulos = datos.isnull().sum()
valores_nulos_ordenados = valores_nulos.sort_values(ascending=False)
porcentaje_nulos = (valores_nulos_ordenados / len(datos)) * 100

# Crear un DataFrame con tres columnas: "Nombre de la Variable", "Valores Nulos", y "Porcentaje de Nulos"
df_valores_nulos = pd.DataFrame({'Nombre de la Variable': valores_nulos_ordenados.index,
                                 'Valores Nulos': valores_nulos_ordenados,
                                 'Porcentaje de Nulos': porcentaje_nulos})

# Filtrar variables con más del 10% de valores nulos
umbral_porcentaje_nulos = 0
df_valores_nulos_filtrado = df_valores_nulos[df_valores_nulos['Porcentaje de Nulos'] > umbral_porcentaje_nulos]


# Ordena el DataFrame por la cantidad de valores nulos
#df_valores_nulos = df_valores_nulos.sort_values(by='Valores Nulos', ascending=False)

# Ordena el DataFrame filtrado por la cantidad de valores nulos
df_valores_nulos_filtrado = df_valores_nulos_filtrado.sort_values(by='Valores Nulos', ascending=False)

# Ajusta el espacio entre los nombres de las variables
plt.figure(figsize=(8, 7))
sns.set(style="whitegrid")
ax = sns.barplot(x="Porcentaje de Nulos", y="Nombre de la Variable", data=df_valores_nulos_filtrado, palette="viridis", orient="h")

# Ajusta el espacio entre los nombres de las variables en el eje y
ax.tick_params(axis='y', which='major', labelsize=8)

plt.title(f"Valores Nulos por Variable (Porcentaje > {umbral_porcentaje_nulos}%)")
#plt.title("Valores Nulos por Variable")
plt.xlabel("Porcentaje de Valores Nulos")
plt.ylabel("Nombre de la Variable")

# Agrega etiquetas de porcentaje a las barras
for p in ax.patches:
    width = p.get_width()
    plt.text(width, p.get_y() + p.get_height() / 2, f'{width:.1f}%', ha='left', va='center')

plt.show()

In [0]:
datos[["marchadod","marchaddo","id_category_path","adelantardo","adelantarod","marchacod","marchacdo"]].head(5)

In [0]:
print("Valores unicos de variable marchadod:",datos["marchadod"].unique()) ,
print("Valores unicos de variable marchaddo:",datos["marchaddo"].unique()) ,
print("Valores unicos de variable id_category_path:",datos["id_category_path"].unique()) ,
print("Valores unicos de variable adelantardo:",datos["adelantardo"].unique()),
print("Valores unicos de variable adelantarod:",datos["adelantarod"].unique()),
print("Valores unicos de variable marchacod:",datos["marchacod"].unique()),
print("Valores unicos de variable marchacdo:",datos["marchacdo"].unique())

### *2.1 TRATAMIENTO DE VALORES NULOS*
- Una vez que ya hemos identificado los valores Nulos pasaremos a tratarlos acontinuacion:

1. Se eliminan las columnas(variables) con mas del 70% de Nulos en sus datos. Esto debido a que con ese porcentaje 
de valores Nulos, no aportan informacion necesaria para hacer el analisis de datos.

In [0]:
#Extraemos los indices(nombres) de las columnas o variables que tienen mas de 70% de nulos
columnas_a_eliminar = porcentaje_nulos[porcentaje_nulos > 70].index
#Eliminamos las variables con mayor porcentaje de valores nulos
datos = datos.drop(columnas_a_eliminar, axis=1)

In [0]:
datos.isnull().sum()

In [0]:
#Eliminamos la variable "arreglolimites", con axis=1 para eliminar la columa, y inplace para eliminar en el actual df
#datos.drop("arreglolimites", axis=1, inplace=True)
#datos.drop(["tiem_creac", "tiem_update"], axis=1, inplace=True)

columnas_a_eliminar = ['columnatestingt', 'marchacod', 'marchacdo','marchadod','marchaddo','id_category_path',
                       'adelantardo','adelantarod','isunderground']

# Filtrar las columnas que existen en el DataFrame
columnas_existentes = [col for col in columnas_a_eliminar if col in datos.columns]

# Verificar si hay columnas para eliminar
if columnas_existentes:
    datos.drop(columnas_existentes, axis=1, inplace=True)

In [0]:
datos.dtypes

In [0]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Supongamos que tienes un DataFrame llamado datos
valores_nulos = datos.isnull().sum()
valores_nulos_ordenados = valores_nulos.sort_values(ascending=False)
porcentaje_nulos = (valores_nulos_ordenados / len(datos)) * 100

# Crear un DataFrame con tres columnas: "Nombre de la Variable", "Valores Nulos", y "Porcentaje de Nulos"
df_valores_nulos = pd.DataFrame({'Nombre de la Variable': valores_nulos_ordenados.index,
                                 'Valores Nulos': valores_nulos_ordenados,
                                 'Porcentaje de Nulos': porcentaje_nulos})

# Filtrar variables con más del 10% de valores nulos
umbral_porcentaje_nulos = 0
df_valores_nulos_filtrado = df_valores_nulos[df_valores_nulos['Porcentaje de Nulos'] >= umbral_porcentaje_nulos]


# Ordena el DataFrame por la cantidad de valores nulos
#df_valores_nulos = df_valores_nulos.sort_values(by='Valores Nulos', ascending=False)

# Ordena el DataFrame filtrado por la cantidad de valores nulos
df_valores_nulos_filtrado = df_valores_nulos_filtrado.sort_values(by='Valores Nulos', ascending=False)

# Ajusta el espacio entre los nombres de las variables
plt.figure(figsize=(8, 15))
sns.set(style="whitegrid")
ax = sns.barplot(x="Porcentaje de Nulos", y="Nombre de la Variable", data=df_valores_nulos_filtrado, palette="viridis", orient="h")

# Ajusta el espacio entre los nombres de las variables en el eje y
ax.tick_params(axis='y', which='major', labelsize=8)

plt.title(f"Valores Nulos por Variable (Porcentaje > {umbral_porcentaje_nulos}%)")
#plt.title("Valores Nulos por Variable")
plt.xlabel("Porcentaje de Valores Nulos")
plt.ylabel("Nombre de la Variable")

# Agrega etiquetas de porcentaje a las barras
for p in ax.patches:
    width = p.get_width()
    plt.text(width, p.get_y() + p.get_height() / 2, f'{width:.1f}%', ha='left', va='center')

plt.show()

### *1.5 IDENTIFICACCION DE VALORES DUPLICADOS*

In [0]:
import pandas as pd
import matplotlib.pyplot as plt

# Supongamos que tienes un DataFrame llamado 'datos'
# Utiliza la función `duplicated` para marcar filas duplicadas
duplicados = datos[datos.duplicated(keep=False)]

# Luego, utiliza `groupby` para contar cuántas veces se repite cada fila duplicada
conteo_duplicados = duplicados.groupby(list(datos.columns)).size().reset_index(name='Cantidad')

# Filtra las filas que se repiten al menos 2 veces
conteo_duplicados = conteo_duplicados[conteo_duplicados['Cantidad'] >= 2]

# Calcular el porcentaje de valores repetidos
cantidad_filas_duplicadas = len(conteo_duplicados)
total_filas = len(datos)
porcentaje_repetidos = (cantidad_filas_duplicadas / total_filas) * 100

# Crear un gráfico de pastel
labels = ['Valores Únicos', 'Valores Repetidos']
sizes = [100 - porcentaje_repetidos, porcentaje_repetidos]
colors = ['lightcoral', 'lightblue']
explode = (0, 0.1)  # Destacar la porción de valores repetidos

plt.pie(sizes, explode=explode, labels=labels, colors=colors, autopct='%1.1f%%', shadow=True, startangle=140)
plt.axis('equal')  # Asegurar que el gráfico sea un círculo

plt.title('Porcentaje de Valores Repetidos en los Datos')

plt.show()

### *2.2 Tratamiento de Valores Duplicados*

In [0]:
#datos = datos.drop_duplicates()

### Manejo de  Variables Categoricas
Metodo Label Encoding

In [0]:
# Define el diccionario de mapeo
map_dict = {True: 1, False: 0}

# Aplica el mapeo a la columna "isload_t"
datos["isload_t"] = datos["isload_t"].map(map_dict)
datos.head()

In [0]:
datos["isload_t"].value_counts()

In [0]:
datos.dtypes

### 2.4 <font color="orange">Renombramos los nombres de las variables para un mejor analisis

In [0]:
# Define un diccionario con los nuevos nombres de las columnas solo para algunas columnas
nuevos_nombres = {'eq_id' : 'id_equipo', 'Equipment' : 'nombre_equipo'}
# Renombra las columnas del DataFrame
datos = datos.rename(columns=nuevos_nombres)
datos.head()

In [0]:
%fs mounts

In [0]:
# Listar todas las bases de datos
spark.sql("SHOW DATABASES").show()

In [0]:
# # n.1 Crear la base de datos si no existe en el almacenamiento de processed (en la ruta donde se almacenaran los datos preprocesados)
spark.sql("CREATE DATABASE IF NOT EXISTS dbproyectocongestion_processed LOCATION '/mnt/datalakemlopsd4m/processed/proyectocongestion_processed/'")

In [0]:
datos.head()

In [0]:
# Renombrar las columnas para eliminar caracteres especiales
datos.columns = [col.replace(" ", "_").replace("-", "_").replace("(", "").replace(")", "") for col in datos.columns]

# Convertir el DataFrame de Pandas a un DataFrame de Spark
spark_datos = spark.createDataFrame(datos)

# Asegurar que las columnas de fecha y hora mantengan sus tipos de datos
from pyspark.sql.functions import col
spark_datos = spark_datos.withColumn("Event_Date", col("Event_Date").cast("timestamp"))
spark_datos = spark_datos.withColumn("instant_date_t", col("instant_date_t").cast("timestamp"))

# Guardar los datos preprocesados en una tabla Delta
spark_datos.write.format("delta").mode("overwrite").saveAsTable("dbproyectocongestion_processed.datapreprocessed_congestion_tabladelta")

In [0]:
import pandas as pd
#1. Leemos los datos de PROCEESED la tabla Delta 
df_delta = spark.read.format("delta").load("/mnt/datalakemlopsd4m/processed/proyectocongestion_processed/datapreprocessed_congestion_tabladelta")
datos = df_delta.toPandas()

In [0]:
datos.head()

In [0]:
datos.dtypes